In [2]:
from statistics import mean
import torch
import numpy as np
import pandas as pd
import pyro
import pyro.distributions as dist
from pyro.infer import Importance, EmpiricalMarginal
import matplotlib.pyplot as plt
%matplotlib inline

pyro.set_rng_seed(101)

##### Instruction:

* Create a simple SCM, with variables x, y, and exogenous variables for x and y, the model should return y and all exogenous values, train a NN that predicts exogenous variable given y as input, preferably non-linear relationship between y and other variables

In [66]:
def SCM_model():
    Nx = pyro.sample("Nx", dist.Uniform(torch.tensor(0.), torch.tensor(1.0)))
    Ny = pyro.sample("Ny", dist.Uniform(torch.tensor(0.), torch.tensor(1.0)))
    
    # using Normal distribution since Delta tends to create issues when using with Importance and EmpiricalMarginal
    x = pyro.sample("x", dist.Normal(Nx, torch.tensor(0.00001)))
    
    # non-linear relationship between x and y
    y = pyro.sample("y", dist.Normal((x**2 + x * Ny + Ny**2), torch.tensor(0.00001)))
    
    return Nx, Ny, y

In [55]:
print(SCM_model())

(tensor(0.8574), tensor(0.8269), tensor(2.1278))


In [58]:
# arbitrarily assigning a value to y
def infer_Nx_Ny(model, y_input, sample_size):
    cond_model = pyro.condition(model, {'y': torch.tensor(y_input)})

    #using Importance sampling
    posterior = pyro.infer.Importance(cond_model, num_samples=sample_size).run()
    Nx_marginal = EmpiricalMarginal(posterior, 'Nx')
    Ny_marginal = EmpiricalMarginal(posterior, 'Ny')

    Nx_samples = [Nx_marginal().item() for _ in range(sample_size)]
    Ny_samples = [Ny_marginal().item() for _ in range(sample_size)]

    return mean(Nx_samples), mean(Ny_samples)

In [65]:
# arbitrarily set y = 1.1
Nx, Ny = infer_Nx_Ny(SCM_model, 1.1, 10000)

print("The predicted Nx is {}, \nand the predicted Ny is {}".format(Nx, Ny))

The predicted Nx is 0.9920719861984253, 
and the predicted Ny is 0.10555022954940796
